In [1]:
from torch import optim
import torch
import torch.utils.data
import argparse
import torch.backends.cudnn as cudnn
import torchvision
import random
import json
import sys
import time
import os
import csv

In [2]:
from Data import cifar10, cifar100, otto
from Net import resnet
from Metrics import metrics
from Losses import loss
from src import train, evaluate

In [3]:
def set_seeds(seed):
    
    random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Обучаем базовую модель `ResNet` без калибровки (`CIFAR10`)

In [ ]:
class args:
    dataset = 'cifar10'
    num_classes = 10
    dataset_root = './data'
    train_batch_size = 128
    test_batch_size = 128
    data_aug = False
    seed = 42
    learning_rate = 0.1
    momentum = 0.9
    optimiser = "sgd"
    loss = "cross_entropy"
    warm_up_epochs = 0
    gpu = True
    loss_mean = False
    load = False

    gamma = 1.0
    gamma2 = 1.0
    gamma3 = 1.0
    lamda = 1.0
    gamma_schedule_step1 = 100
    gamma_schedule_step2 = 250
    temperature = 1.0
    bsce_norm = 1
    num_bins = 15

    weight_decay = 5e-4
    log_interval = 50
    save_interval = 50
    save_loc = './model/'
    model = "resnet50"
    epoch = 350
    first_milestone = 150 #Milestone for change in lr
    second_milestone = 250 #Milestone for change in lr
    nesterov = False

In [5]:
set_seeds(args.seed)

In [6]:
train_loader, val_loader = cifar10.get_train_valid_loader(
    batch_size=args.train_batch_size,
    augment=args.data_aug,
    random_seed=args.seed,
    valid_size=0.1,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

test_loader = cifar10.get_test_loader(
    batch_size=args.test_batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
save_path = os.path.join(args.save_loc, args.dataset, args.model)
model_name = args.dataset+'-'+args.model+'-'+args.loss
save_loc = os.path.join(save_path, model_name, str(args.seed))

os.makedirs(os.path.join(save_loc, 'csv'), exist_ok=True)
csv_file_path = os.path.join(save_loc, 'csv')
csv_file = model_name + '_' + '.csv'

In [8]:
cuda = args.gpu
if (torch.cuda.is_available() and cuda):
    cuda = True
    
device = torch.device("cuda" if cuda else "cpu")
print("CUDA set: " + str(cuda))


model = resnet.resnet50(num_classes=args.num_classes)
model.cuda()
model = torch.nn.DataParallel(
    model, device_ids=range(torch.cuda.device_count()))
cudnn.benchmark = True

opt_params = model.parameters()
optimizer = optim.SGD(opt_params,
                        lr=args.learning_rate,
                        momentum=args.momentum,
                        weight_decay=args.weight_decay,
                        nesterov=args.nesterov)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[args.first_milestone, args.second_milestone], gamma=0.1)

loss_function = loss.cross_entropy({}, device)

CUDA set: True


In [9]:
mode = 'a' if args.load else 'w'
with open(csv_file_path+'/'+csv_file, mode=mode, newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["args"] + [f"{key}={value}" for key, value in vars(args).items()])
    writer.writerow(["epoch", "train_loss", "val_loss", "val_acc", "val_ece", "test_acc", "test_ece"])

In [10]:
start_epoch = 0
num_epochs = args.epoch


In [ ]:
for epoch in range(start_epoch, num_epochs):
    train_loss, _, labels_list, fulldataset_logits, predictions_list, confidence_list = train.train_single_epoch_warmup(args,
                                    epoch,
                                    model,
                                    train_loader,
                                    val_loader,
                                    optimizer,
                                    device,
                                    loss_function=loss_function,
                                    num_labels=args.num_classes,
                                    )
    scheduler.step()

    (val_loss, val_confusion_matrix, val_acc, val_ece, val_bin_dict,
    val_adaece, val_adabin_dict, val_mce, val_classwise_ece, val_classwise_dict, val_logits, val_labels) = evaluate.evaluate_dataset(model, val_loader, device, num_bins=args.num_bins, num_labels=args.num_classes)

    (test_loss, test_confusion_matrix, test_acc, test_ece, test_bin_dict, 
    test_adaece, test_adabin_dict, test_mce, test_classwise_ece, test_classwise_dict, test_logits, test_labels) = evaluate.evaluate_dataset(model, test_loader, device, num_bins=args.num_bins, num_labels=args.num_classes)

    with open(csv_file_path+'/'+csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch, train_loss, val_loss, val_acc, val_ece, test_acc, test_ece])

Train Epoch: 0 [0/45056 (0%)]	Loss: 334.948608
Train Epoch: 0 [1280/45056 (3%)]	Loss: 311.875549
Train Epoch: 0 [2560/45056 (6%)]	Loss: 366.561859
Train Epoch: 0 [3840/45056 (9%)]	Loss: 355.308167
Train Epoch: 0 [5120/45056 (11%)]	Loss: 363.199280
Train Epoch: 0 [6400/45056 (14%)]	Loss: 320.914093
Train Epoch: 0 [7680/45056 (17%)]	Loss: 365.231018
Train Epoch: 0 [8960/45056 (20%)]	Loss: 306.125854
Train Epoch: 0 [10240/45056 (23%)]	Loss: 341.752258
Train Epoch: 0 [11520/45056 (26%)]	Loss: 282.378174
Train Epoch: 0 [12800/45056 (28%)]	Loss: 262.509430
Train Epoch: 0 [14080/45056 (31%)]	Loss: 275.806488
Train Epoch: 0 [15360/45056 (34%)]	Loss: 247.005341
Train Epoch: 0 [16640/45056 (37%)]	Loss: 265.180389
Train Epoch: 0 [17920/45056 (40%)]	Loss: 224.759750
Train Epoch: 0 [19200/45056 (43%)]	Loss: 238.795242
Train Epoch: 0 [20480/45056 (45%)]	Loss: 276.470306
Train Epoch: 0 [21760/45056 (48%)]	Loss: 250.555420
Train Epoch: 0 [23040/45056 (51%)]	Loss: 236.141663
Train Epoch: 0 [24320/45056

In [ ]:
start_epoch = epoch
for epoch in range(0, start_epoch):
    scheduler.step()

In [6]:
KAG_CFG_DIR = "./.kaggle" 

train_loader, val_loader = otto.get_train_valid_loader(
    batch_size=256,
    random_seed=42,
    valid_size=0.1,
    kaggle_config_dir=KAG_CFG_DIR,
)

test_loader = otto.get_test_loader(
    batch_size=256,
    kaggle_config_dir=KAG_CFG_DIR,
)

[otto] CSV files not found – downloading from Kaggle …



100%|██████████| 6.05M/6.05M [00:00<00:00, 1.14GB/s]
